# Project A14 KAGGLE - A MOVIE SCORE PREDICTION
## Predicting IMDb movie ratings from given parameters
Marten Mark, Karl Soosalu, Mark Alexander


# Preprocessing

In [36]:
# Imports

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error

from keras.layers import Dense, BatchNormalization
from keras.models import Sequential
from keras.optimizers import SGD

In [2]:
movies = pd.read_csv("IMDb movies.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Converting date_published into year, month and day
movies["date_published"].iloc[83917] = "2019" #Replacing anomaly
date = lambda x: pd.Series([i for i in x.split('-')])
newCols = movies['date_published'].apply(date)
newCols = newCols.fillna(0)

movies["year_new"] = newCols[0]
movies["month"] = newCols[1]
movies["day"] = newCols[2]

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [5]:
# Removing columns
movies = movies.drop(["imdb_title_id", "title", "original_title", "year", "description", "budget", "usa_gross_income", "worlwide_gross_income", "metascore", "votes", "reviews_from_users", "reviews_from_critics", "date_published"], axis=1)

In [6]:
movies["language"] = movies.language.replace("None", np.nan)

In [7]:
movies = pd.concat([movies.drop('language', 1), movies['language'].str.get_dummies(sep=", ")], 1)
movies = pd.concat([movies.drop('genre', 1), movies['genre'].str.get_dummies(sep=", ")], 1)
movies = pd.concat([movies.drop('country', 1), movies['country'].str.get_dummies(sep=", ")], 1)

In [8]:
movies = movies.drop(["production_company", "actors", "director", "writer"], axis=1)

In [76]:
X = movies.drop("avg_vote", axis=1)
y = movies.avg_vote

X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.3, random_state=0)

In [66]:
linear = LinearRegression().fit(X_train, y_train)
pred_linear = linear.predict(X_test)
pred[pred > 10] = 10
pred[pred < 0] = 0

In [67]:
print(mean_absolute_error(y_test, pred))

0.7721142269857987


In [69]:
mdl = Sequential()
mdl.add(Dense(64, activation="tanh", input_shape=(X_train.shape[1],)))
mdl.add(BatchNormalization())
mdl.add(Dense(32, activation="tanh"))
mdl.add(BatchNormalization())
mdl.add(Dense(16, activation="selu"))
mdl.add(BatchNormalization())
mdl.add(Dense(7, activation="linear"))
mdl.add(Dense(1, activation="linear"))
mdl.compile(loss="mean_squared_error", optimizer="sgd")

In [86]:
y_train = np.asarray(y_train).astype(np.float32)
X_train = np.asarray(X_train).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)

mdl.fit(X_train, y_train, epochs=4, validation_data=(X_test, y_test), batch_size=256, verbose=True) #, validation_data=(X_test, y_test)


Epoch 1/4
235/235 [==============================] - 1s 4ms/step - loss: 1.6274 - val_loss: 2.5846
Epoch 2/4
235/235 [==============================] - 1s 4ms/step - loss: 1.6274 - val_loss: 2.6745
Epoch 3/4
235/235 [==============================] - 1s 4ms/step - loss: 1.6274 - val_loss: 2.5137
Epoch 4/4
235/235 [==============================] - 1s 4ms/step - loss: 1.6272 - val_loss: 2.6376
